In [8]:
import requests
import boto3
import os

def download_stock_data(ticket, date_from, date_to, granularity, bucket_name, s3_path):
    # Base URL for the stock data
    base_url = "https://stooq.pl/q/d/l/"

    # Parameters for the request
    params = {
        's': ticket,
        'd1': date_from,
        'd2': date_to,
        'i': granularity
    }

    # Make the GET request to download the data
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Define the local file name based on the parameters
        file_name = f"{ticket}_{date_from}_{date_to}_{granularity}.csv"
        
        # Save the content to a local file
        with open(file_name, 'wb') as file:
            file.write(response.content)
        
        print(f"Data downloaded and saved as {file_name}")

        # Upload the file to S3
        s3_client = boto3.client('s3')
        s3_key = os.path.join(s3_path, file_name)
        s3_client.upload_file(file_name, bucket_name, s3_key)
        
        print(f"File uploaded to s3://{bucket_name}/{s3_key}")

        # Clean up the local file if needed
        os.remove(file_name)
    else:
        print(f"Failed to download data. HTTP Status Code: {response.status_code}")

# Example usage
bucket_name = 'sagemaker-bucket-ds'
folder_name = '01_STOCKS/DATA/RAW'
date_start = '20000101'
date_end = '20231231'
tickers = ['^spx', "msft.us", "aapl.us", "mcd.us"]
for t in tickers:
    download_stock_data(t, date_start, date_end, 'd', bucket_name, folder_name)

Data downloaded and saved as ^spx_20000101_20231231_d.csv
File uploaded to s3://sagemaker-bucket-ds/01_STOCKS/DATA/RAW/^spx_20000101_20231231_d.csv
Data downloaded and saved as msft.us_20000101_20231231_d.csv
File uploaded to s3://sagemaker-bucket-ds/01_STOCKS/DATA/RAW/msft.us_20000101_20231231_d.csv
Data downloaded and saved as aapl.us_20000101_20231231_d.csv
File uploaded to s3://sagemaker-bucket-ds/01_STOCKS/DATA/RAW/aapl.us_20000101_20231231_d.csv
Data downloaded and saved as mcd.us_20000101_20231231_d.csv
File uploaded to s3://sagemaker-bucket-ds/01_STOCKS/DATA/RAW/mcd.us_20000101_20231231_d.csv
